In [1]:
from dep_tools.loaders import Sentinel2OdcLoader
from dep_tools.namers import LocalPath
from dep_tools.runner import run_by_area_dask_local

# TODO: Fix up the local writer to handle
# multiple-file writing...
from dep_tools.writers import LocalDsWriter
from logging import Logger


from src.run_task import MangrovesProcessor

In [2]:
from src.grid import grid
datetime = "2023"


processor = MangrovesProcessor()

writer = LocalDsWriter(itempath=LocalPath(
    sensor="s2",
    dataset_id="mangroves",
    version="local_test_1",
    time=datetime,
    local_folder="data/test_mangroves"))

In [4]:
region_code = "FJ"
region_index = "006"

areas = grid[grid.index == (region_code, region_index)]

loader = Sentinel2OdcLoader(
    epsg=3832,
    datetime=datetime,
    dask_chunksize=dict(band=1, time=1, x=4096, y=4096),
    odc_load_kwargs=dict(fail_on_error=False, resolution=10, bands=["B04", "B08"]),
)

run_by_area_dask_local(
    areas=areas,
    loader=loader,
    processor=processor,
    writer=writer,
    logger=Logger("test_mangroves"),
    continue_on_error=False,
    local_cluster_kwargs={},
)

  0%|                                                                                                                                                             | 0/1 [00:00<?, ?it/s]/home/jesse/Projects/D4D/dep-mangroves/venv/lib/python3.10/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/home/jesse/Projects/D4D/dep-mangroves/venv/lib/python3.10/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/home/jesse/Projects/D4D/dep-mangroves/venv/lib/python3.10/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/home/jesse/Projects/D4D/dep-mangroves/venv/lib/python3.10/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The 